# Equal Weight S&P 500
https://www.youtube.com/watch?v=xfzGZB4HhEE&ab_channel=freeCodeCamp.org

you actually never followed that yt video at all.

In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime

# preamble

- [x] establish api key
- [x] establish variables
- [x] get symbols for all sp500 
- [x] run test query to confirm errorless execution

In [ ]:
# establish api key
keys_file = open("alphaVantage.txt")
api_key = keys_file.readlines()[0]

In [ ]:
#S&P 500 symbols
sp500_symbols = pd.read_csv('https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv')['Symbol']

In [ ]:
# # # test query. if things break, see if this is still working
# ticker = 'ABMD'
# interval = '5min'
# function = 'TIME_SERIES_INTRADAY'
# # run test query
# ibm = requests.get(f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={api_key}').json()
# # aapl

# build dataframe of 5 minute moves for yesterday

In [ ]:
bandit = list(sp500_symbols)#[:6]
#bandit

In [ ]:
def get_5min(ticker, interval, function):
    thing = {}
    
    for i in ticker:
        try:
            ticker = str(i)
            function = str(function)
            api_info = requests.get(f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={api_key}').json()
            shortcode = api_info['Meta Data']['2. Symbol'] # get the symbol e.g. "AAPL"
            five_min = api_info['Time Series (5min)'] # get the 5 minute intervals THIS WILL BREAK IF YOU CHANGE THE FUNCTION
            n = api_info['Time Series (5min)'] # get the 5 minute intervals

            close_price = {} #build dictionary of times and prices
            for i in five_min:
                close_price[i.split(' ')[1]] = five_min[i]['4. close']

            out = {shortcode: close_price}
            thing.update(out)
            time.sleep(13)
    
        except:
            continue

        df = pd.DataFrame.from_dict(thing,orient='index').T
        df.sort_index(inplace=True)

    return df

In [ ]:
df_idk = get_5min(bandit, '5min', 'TIME_SERIES_INTRADAY')
df_idk

In [ ]:
# backfill, forwardfill, save to csv
today = datetime.today().strftime('%Y-%m-%d')
df_idk = df_idk.ffill().bfill()
df_idk.to_csv(f'./data/raw/sp500-{today}.csv')